In [1]:
import re
import os
import json
import pandas

import importlib
import aux_qa
importlib.reload(aux_qa)

<module 'aux_qa' from '/.auto/home6/mroemmele/CoreNLP/python/question_generation/aux_qa.py'>

## Evaluate generated questions with QA model

Objective is for QA model to predict correct answers to generated questions

In [2]:
'''Specify which set of generated questions to evaluate on'''

output_dir = '/home/mroemmele/question_generation/squad_newsqa_gpt2_tok_no_masking_models/baseline/outputs/'
gen_questions_filepath = "squad_newsqa_test_set_detok.txt"

gen_questions = [question.strip() for question in open(os.path.join(output_dir, gen_questions_filepath))]
gen_questions[-10:]

["What was the name of John's Calvert Island?",
 "What was the name of John and Elizabeth's life?",
 'When was she added?',
 'Who is Nancy Cappelmann?',
 'What did he say?',
 'How many died?',
 'Where are the happened?',
 'Where is the match?',
 'What did the cause of the deaths of deaths?',
 'How many horses dropped dead?']

In [3]:
'''Load test data'''

input_texts_filepath = "/home/mroemmele/question_generation/squad_newsqa_untok_data/test/paragraphs.txt"
gold_questions_filepath = "/home/mroemmele/question_generation/squad_newsqa_untok_data/test/questions.txt"
gold_answers_filepath = "/home/mroemmele/question_generation/squad_newsqa_untok_data/test/answers_only.txt"

# For QA, mask answer annotations in inputs
answer_annotated_input_texts = [text.strip() for text in open(input_texts_filepath)]
input_texts = [re.sub("\s?</?ANSWER>", "", text.strip()) for text in answer_annotated_input_texts]
gold_questions = [question.strip() for question in open(gold_questions_filepath)]
#Test set consists of duplicate questions which could have more than one correct answer.
gold_answers = [answer.strip().split("\t") for answer in open(gold_answers_filepath)]
#Map each question to answer set so all answers can be taken into account during evaluation.
assert len(input_texts) == len(gold_questions) == len(gold_answers)
pandas.DataFrame({'input_text': input_texts, 'gold_question': gold_questions, 'gold_answer': gold_answers})[:10]

,input_text,gold_question,gold_answer
0,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,[Denver Broncos]
1,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...,[Carolina Panthers]
2,Super Bowl 50 was an American football game to...,Where did Super Bowl 50 take place?,"[Santa Clara, California, Levi's Stadium, Levi..."
3,Super Bowl 50 was an American football game to...,Which NFL team won Super Bowl 50?,[Denver Broncos]
4,Super Bowl 50 was an American football game to...,What color was used to emphasize the 50th anni...,[gold]
5,Super Bowl 50 was an American football game to...,What was the theme of Super Bowl 50?,"[""golden anniversary"", ""golden anniversary, go..."
6,Super Bowl 50 was an American football game to...,What day was the game played on?,"[February 7, February 7, 2016]"
7,Super Bowl 50 was an American football game to...,What is the AFC short for?,[American Football Conference]
8,Super Bowl 50 was an American football game to...,What was the theme of Super Bowl 50?,"[""golden anniversary"", gold, gold-themed]"
9,Super Bowl 50 was an American football game to...,What does AFC stand for?,[American Football Conference]


In [5]:
'''Run QA on generated questions'''

batch_size = 10
pred_answers = []
for batch_idx in range(0, len(input_texts), batch_size):
    batch_pred_answers = [answer.strip() for answer in 
                          aux_qa.answer_questions(input_texts[batch_idx:batch_idx + batch_size], 
                                                  gen_questions[batch_idx:batch_idx + batch_size])]
    pred_answers.extend(batch_pred_answers)
    print(batch_idx)
    #if batch_idx and batch_idx % 1024 == 0:
    #     break

> /.auto/home6/mroemmele/CoreNLP/python/question_generation/aux_qa.py(29)answer_questions()
-> answers = br.batch_inference(questions, texts, model, tokenizer, device, MAX_SEQ_LENGTH, batch_size)
(Pdb) n
Done batch:  0
> /.auto/home6/mroemmele/CoreNLP/python/question_generation/aux_qa.py(30)answer_questions()
-> answers = [answer[0] if answer[0] != 'Answer not found' else ""
(Pdb) answers
['Denver Broncos', 'Carolina Panthers', "Levi's Stadium", 'Denver Broncos', '2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016,', 'Super Bowl 50', 'February 7, 2016,', 'Carolina Panthers', 'Super Bowl 50', 'Carolina Panthers']
(Pdb) q


BdbQuit: 

In [16]:
'''Evaluate answers'''

n_answers = None
assert len(pred_answers[:n_answers]) == len(gold_answers[:n_answers])
# import pdb;pdb.set_trace()
eval_scores = aux_qa.evaluate(pred_answers[:n_answers], gold_answers[:n_answers])
eval_scores

--Return--
> <ipython-input-16-b43342cf34e8>(4)<module>()->None
-> import pdb;pdb.set_trace()
(Pdb) b 5
Breakpoint 5 at <ipython-input-16-b43342cf34e8>:5
(Pdb) c
> <ipython-input-16-b43342cf34e8>(5)<module>()->None
-> eval_scores = aux_qa.evaluate(pred_answers, gold_answers[:10])
(Pdb) s
--Call--
> /.auto/home6/mroemmele/CoreNLP/sandbox/question_generation/aux_qa.py(59)evaluate()
-> def evaluate(pred_answers, all_correct_answers):
(Pdb) r
--Return--
> /.auto/home6/mroemmele/CoreNLP/sandbox/question_generation/aux_qa.py(82)evaluate()->{'f1': 0.7, 'precision': 0.7, 'recall': 0.7}
-> return {'precision': mean_precision, 'recall': mean_recall, 'f1': mean_f1}
(Pdb) precision_scores
[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]
(Pdb) pred_answers[3]
'Denver Broncos'
(Pdb) all_correct_answers[3]
['Carolina Panthers']
(Pdb) pred_answers[4]
'Denver Broncos'
(Pdb) pred_answers[5]
'Denver Broncos'
(Pdb) all_correct_answers[5]
['Carolina Panthers']
(Pdb) c


{'precision': 0.7, 'recall': 0.7, 'f1': 0.7}

In [ ]:
'''Print sample of output'''

for (input_text, gen_question, pred_answer, 
     gold_question, gold_answer) in zip(answer_annotated_input_texts[:1000], gen_questions[:1000], pred_answers[:1000], 
                                        gold_questions[:1000], gold_answers[:1000]):
    print("TEXT:", input_text)
    print("GEN QUESTION:", gen_question)
    print("PRED ANSWER:", pred_answer)
    print("HUMAN QUESTION:", gold_question)
    print("CORRECT ANSWER:", gold_answer)
    print("")

In [ ]:
'''Create directory for QA evaluation output'''

if not os.path.isdir(os.path.join(output_dir, 'qa_eval')):
    os.mkdir(os.path.join(output_dir, 'qa_eval'))

In [ ]:
'''Save full visualization of output'''

with open(os.path.join(output_dir, "qa_eval/squad_test_full_qa_output_paragraphs.txt"), 'w') as f:
    for (input_text, gen_question, pred_answer, 
         gold_question, gold_answer) in zip(answer_annotated_input_texts, gen_questions, pred_answers, 
                                            gold_questions, gold_answers):
        
        f.write("TEXT: {}\n".format(input_text))
        f.write("GEN QUESTION: {}\n".format(gen_question))
        f.write("PRED ANSWER: {}\n".format(pred_answer))
        f.write("HUMAN QUESTION: {}\n".format(gold_question))
        f.write("CORRECT ANSWER: {}\n".format(gold_answer))
        f.write("\n")

In [ ]:
'''Save predicted answers'''

with open(os.path.join(output_dir, "qa_eval/squad_test_pred_answers_paragraphs.txt"), 'w') as f:
    f.write("\n".join(pred_answers))

In [ ]:
'''Save evaluation scores'''

with open(os.path.join(output_dir, "qa_eval/squad_test_eval_scores_paragraphs.json"), 'w') as f:
    json.dump(eval_scores, f)